In [6]:
import io
from tkinter import Tk, Button, filedialog, Label, Checkbutton, BooleanVar
from PIL import Image, ImageTk
from rembg import remove
from tkinter import ttk

def upload_image():
    global image_path
    image_path = filedialog.askopenfilename()

def upload_background():
    global background_path
    background_path = filedialog.askopenfilename()

def process_images():
    global result_label
    global add_background_var
    if not image_path:
        print("Please upload an image.")
        return
    
    with open(image_path, 'rb') as f_input:
        input_data = f_input.read()
        output_data = remove(input_data)
        image = Image.open(io.BytesIO(output_data))
        
        if add_background_var.get():
            if not background_path:
                print("Please upload a background image.")
                return
            with open(background_path, 'rb') as f_background:
                background_data = f_background.read()
                background = Image.open(io.BytesIO(background_data))
                image = image.resize(background.size)
                result = Image.alpha_composite(background.convert("RGBA"), image.convert("RGBA"))
        else:
            result = image

        # Convert result image to Tkinter PhotoImage
        result_photo = ImageTk.PhotoImage(result)

        # Update the label with the new result image
        result_label.configure(image=result_photo)
        result_label.image = result_photo

        # Store result image
        global processed_image
        processed_image = result

def save_image():
    if processed_image:
        file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
        if file_path:
            processed_image.save(file_path)

# Create the GUI window
root = Tk()
root.title("Image Background Remover")

# Set custom style for buttons
style = ttk.Style()
style.configure('Cute.TButton', foreground='black', background='#FFD700', font=('Arial', 12, 'bold'))

# Frame to hold buttons
frame = ttk.Frame(root)
frame.pack(padx=20, pady=20)

# Button for uploading image
upload_image_button = ttk.Button(frame, text="Upload Image", command=upload_image, style='Cute.TButton')
upload_image_button.grid(row=0, column=0, padx=5, pady=5)

# Button for uploading background
upload_background_button = ttk.Button(frame, text="Upload Background", command=upload_background, style='Cute.TButton')
upload_background_button.grid(row=0, column=1, padx=5, pady=5)

# Checkbutton for adding background
add_background_var = BooleanVar()
add_background_checkbutton = Checkbutton(frame, text="Add Background", variable=add_background_var)
add_background_checkbutton.grid(row=1, columnspan=2, padx=5, pady=5)

# Button for processing images
process_button = ttk.Button(frame, text="Process Images", command=process_images, style='Cute.TButton')
process_button.grid(row=2, columnspan=2, padx=5, pady=5)

# Button for saving image
save_button = ttk.Button(frame, text="Save Image", command=save_image, style='Cute.TButton')
save_button.grid(row=3, columnspan=2, padx=5, pady=5)

# Label to display the result image
result_label = Label(root)
result_label.pack(pady=5)

root.mainloop()
